# 目标检测训练

In [1]:
#!/usr/bin/env python3
# coding=utf-8
import os
import time

from iobjectspy import open_datasource
from iobjectspy.ml.vision import Trainer, DataPreparation, Inference

Using TensorFlow backend.


## 设置TIF影像,矢量标签路径

In [2]:
curr_dir = ''
data_dir = os.path.join(curr_dir, '..','..','example_data')
input_data = os.path.join(data_dir, 'training','plane.tif')
ds = open_datasource(os.path.join(data_dir, 'training','label.udbx'))

java -cp /home/data/hou/workspaces/iobjectspy/venv/lib/python3.6/site-packages/iobjectspy-10.0.0-py3.6.egg/iobjectspy/_jsuperpy/jars/iobjects-py4j.jar com.supermap.jsuperpy.ApplicationExample 127.0.0.1 47659
[iObjectsPy]: Connection gateway-service successful, Python callback port bind 40065



## 设置训练数据生成路径

In [3]:
output_path = os.path.join(curr_dir, '..','..','out')

## 生成VOC格式的训练数据

In [4]:
input_label = ds['label']
label_class_field = None
output_name = 'VOC'
start_time = time.time()
DataPreparation.create_training_data(input_data=input_data, input_label=input_label,
                                     label_class_field=label_class_field, output_path=output_path,
                                     output_name=output_name,
                                     training_data_format='VOC', tile_format='jpg', tile_size_x=512,
                                     tile_size_y=512, tile_offset_x=256,
                                     tile_offset_y=256, tile_start_index=0, save_nolabel_tiles=False)

print('完成，共耗时{}s，训练数据保存在 {}'.format(
    time.time() - start_time, os.path.join(output_path, output_name)))

train data saved to `../../out/VOC`
完成，共耗时3.8450088500976562s，训练数据保存在 ../../out/VOC


## 设置预训练模型路径

In [5]:
backbone_weight_path = os.path.join(curr_dir, '..','..','backbone','res101.ckpt')

## 设置模型配置文件

In [6]:
config = os.path.join(curr_dir, '..','..','trainer_config','object_detection_train_config.sdt')

## 设置训练数据输入路径

In [7]:
train_data_path = os.path.join(curr_dir, '..','..','out','VOC')

## 设置输出路径

In [8]:
#日志输出路径
log_path = os.path.join(curr_dir, '..','..','out','log')
#模型输出路径
output_model_path = os.path.join(curr_dir, '..','..','out','object_detection_model')
output_model_name = 'saved_model'

## 基于Faster R-CNN的目标检测训练
通过利用训练数据制作出来的VOC数据集进行目标检测模型训练,得到可用于预测的模型文件

In [ ]:
start_time = time.time()

Trainer(train_data_path=train_data_path, config=config, epoch=1, batch_size=1, lr=0.001,
        output_model_path=output_model_path,
        output_model_name=output_model_name, backbone_name='res101',
        backbone_weight_path=backbone_weight_path, log_path=log_path, reload_model=False).object_detect_train()

print('完成，共耗时{}s，模型文件保存在 {}'.format(
    time.time() - start_time, os.path.join(output_model_path, output_model_name)))

Output snapshot model will be saved to `../../out/log/ckpt_model_path`
TensorFlow summaries will be saved to `../../out/log/tensorflow`
Output model will be saved to `../../out/object_detection_model/saved_model`
Appending horizontally-flipped training examples...
done
Preparing training data...
done
Preparing training data...
done
Filtered 30 roidb entries: 100 -> 70
Filtered 2 roidb entries: 7 -> 5
Solving...


/home/data/hou/workspaces/iobjectspy/venv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Loading initial model weights from ../../backbone/res101.ckpt
Loaded.
Fix Resnet V1 layers..
Fixed.
iter: 10 / 50, total loss: 0.752909
 >>> rpn_loss_cls: 0.291161
 >>> rpn_loss_box: 0.054692
 >>> loss_cls: 0.024543
 >>> loss_box: 0.000000
 >>> lr: 0.001000
speed: 7.797s / iter
iter: 20 / 50, total loss: 1.523404
 >>> rpn_loss_cls: 0.262039
 >>> rpn_loss_box: 0.060998
 >>> loss_cls: 0.366770
 >>> loss_box: 0.451084
 >>> lr: 0.001000
speed: 7.445s / iter
iter: 30 / 50, total loss: 1.527546
 >>> rpn_loss_cls: 0.053517
 >>> rpn_loss_box: 0.042356
 >>> loss_cls: 0.490019
 >>> loss_box: 0.559145
 >>> lr: 0.001000
speed: 7.528s / iter


## 基于目标检测的遥感飞机影像的预测

In [ ]:
"""
    目标检测类型支持 'plane', 'ship', 'tennis-court', 'vehicle'
"""
input_data = os.path.join(data_dir, 'inference','plane_infer.tif')
model_path = os.path.join(curr_dir, '..','..','model','plane','plane.sdm')
out_data = os.path.join(curr_dir, '..','..','out','plane.udbx')
category_name = "plane"

start_time = time.time()

Inference(input_data=input_data, model_path=model_path, out_data=out_data,
              out_dataset_name='out_plane').object_detect_infer(
        category_name=category_name, nms_thresh=0.3, score_thresh=0.3)

print('完成，共耗时{}s，预测结果保存在 {}'.format(
    time.time() - start_time, os.path.join(out_data)))
